### AB Model from spec

Package installation

In [1]:
# !pip3 install seaborn
# !pip3 install python-statemachine
# !pip3 install mesa
# !pip3 install transitions
# !pip3 install scipy
# !pip3 install cufflinks
# !pip3 install graphviz pygraphviz
# !pip3 install graphviz
# !pip3 install transitions[diagrams]
# !pip3 uninstall mesa

Package importation

In [2]:
# imports
import os
import seaborn as sns
from random import choice
import warnings
warnings.simplefilter("ignore")
import pandas as pd
import numpy as np
import mesa
from mesa import Agent, Model
from mesa.time import RandomActivation, RandomActivationByType, SimultaneousActivation
from mesa.datacollection import DataCollector
from matplotlib import pyplot as plt, patches
import scipy.stats as ss
import cufflinks as cf
cf.go_offline()
from plotly.offline import iplot
from transitions import Machine
import random
from transitions.extensions import GraphMachine
import graphviz
import timeit
from datetime import datetime
import logging
# os.environ["PATH"] += os.pathsep + '/Users/ia329/homebrew/bin' # for graphviz

Model component importation

In [4]:
import config.model_config as cfg
from EV.agent import EV, ChargeStation
import EV.model as model
from EV.statemachine import EVSM, LSM
from EV.model import get_evs_charge, get_evs_charge_level, get_evs_active, get_evs_queue, get_evs_travel, get_evs_not_idle, get_active_chargestations, get_eod_evs_socs, get_evs_destinations, get_ev_distance_covered
# old:  get_evs_charging,

State diagram

### Model

Station config import

In [7]:
import csv

def nest_charging_stations(csv_file):
    nested_dict = {}
    
    with open(csv_file, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        
        for row in reader:
            route = row['Route']
            station = row['Station']
            cpid = row['CPID']
            power = int(row['Power'])
            distance = int(row['Distance'])
            price = float(row['Price'])
            green = bool(int(row['Green']))
            booking = bool(int(row['Booking']))
            
            if route not in nested_dict:
                nested_dict[route] = {}
                
            if station not in nested_dict[route]:
                nested_dict[route][station] = {}
                
            nested_dict[route][station]['CPID'] = cpid
            nested_dict[route][station]['Power'] = power
            nested_dict[route][station]['Distance'] = distance
            nested_dict[route][station]['Price'] = price
            nested_dict[route][station]['Green'] = green
            nested_dict[route][station]['Booking'] = booking
            
    return nested_dict


In [8]:
station_config = nest_charging_stations(cfg.STATION_PATH +'stations.csv')

In [10]:
print(station_config)

{'A-B': {'1': {'CPID': 'CS_AB_1_3', 'Power': 60, 'Distance': 40, 'Price': 10.0, 'Green': False, 'Booking': False}, '2': {'CPID': 'CS_AB_2_2', 'Power': 7, 'Distance': 20, 'Price': 6.0, 'Green': False, 'Booking': False}, '3': {'CPID': 'CS_AB_3_3', 'Power': 60, 'Distance': 50, 'Price': 10.0, 'Green': False, 'Booking': False}, '4': {'CPID': 'CS_AB_4_3', 'Power': 60, 'Distance': 10, 'Price': 10.0, 'Green': False, 'Booking': False}, '5': {'CPID': 'CS_AB_5_3', 'Power': 7, 'Distance': 30, 'Price': 6.0, 'Green': False, 'Booking': False}}, 'A-C': {'6': {'CPID': 'CS_AC_6_2', 'Power': 7, 'Distance': 40, 'Price': 6.0, 'Green': False, 'Booking': False}, '7': {'CPID': 'CS_AC_7_2', 'Power': 7, 'Distance': 40, 'Price': 6.0, 'Green': False, 'Booking': False}, '8': {'CPID': 'CS_AC_8_2', 'Power': 7, 'Distance': 40, 'Price': 6.0, 'Green': False, 'Booking': False}, '9': {'CPID': 'CS_AC_9_2', 'Power': 7, 'Distance': 40, 'Price': 6.0, 'Green': False, 'Booking': False}}}


In [ ]:
{
    'A-B': 
    {'1': {'CPID': 'CS_AB_1_3', 'Power': 60, 'Distance': 40, 'Price': 10.0, 'Green': False, 'Booking': False}, 
    '2': {'CPID': 'CS_AB_2_2', 'Power': 7, 'Distance': 20, 'Price': 6.0, 'Green': False, 'Booking': False}, 
    '3': {'CPID': 'CS_AB_3_3', 'Power': 60, 'Distance': 50, 'Price': 10.0, 'Green': False, 'Booking': False}, 
    '4': {'CPID': 'CS_AB_4_3', 'Power': 60, 'Distance': 10, 'Price': 10.0, 'Green': False, 'Booking': False}, 
    '5': {'CPID': 'CS_AB_5_3', 'Power': 7, 'Distance': 30, 'Price': 6.0, 'Green': False, 'Booking': False}
  }, 
  'A-C': 
        {'6': {'CPID': 'CS_AC_6_2', 'Power': 7, 'Distance': 40, 'Price': 6.0, 'Green': False, 'Booking': False}, 
         '7': {'CPID': 'CS_AC_7_2', 'Power': 7, 'Distance': 40, 'Price': 6.0, 'Green': False, 'Booking': False}, 
         '8': {'CPID': 'CS_AC_8_2', 'Power': 7, 'Distance': 40, 'Price': 6.0, 'Green': False, 'Booking': False}, 
         '9': {'CPID': 'CS_AC_9_2', 'Power': 7, 'Distance': 40, 'Price': 6.0, 'Green': False, 'Booking': False}
         }
         
}

In [ ]:
nest_charging_stations(cfg.STATION_PATH +'stations.csv')

Model parameters

In [4]:
# 365 days = 8760 hours
# 30 days = 720 hours
# 2 days = 48 hours
# 7 days = 168 hours
# ticks =  48 # 3600 ticks = 3600 seconds = 1 hour
# no_evs = 3

# no_css = 5
# no_cps = 2
# check points = [40, 80, 120, 160, 200, 240, 280]
# # no of css must be fixed
# '40':2, '80':3, '120':4, '160':2, '200':2
ticks = 48
no_evs = 20
params = {'no_css': 5, 
          'no_cps_per_cs':{
                '1':2, '2':3, '3':4, '4':2, '5':2
                },
            'default_cppcs': 3,
            }


In [5]:
# model_run = model.EVModel(ticks=ticks, no_evs=no_evs, no_css=no_css, no_cps=no_cps)
# for i in range(ticks):
#     model_run.step()
model_run = model.EVModel(ticks=ticks, no_evs=no_evs, params=params)
for i in range(ticks):
    model_run.step()

Creating agents...

CS info: ID: 0, initialized. Charge rate: 7.5 kW.
It has 3 charging points.

CS info: ID: 1, initialized. Charge rate: 7.5 kW.
It has 3 charging points.

CS info: ID: 2, initialized. Charge rate: 7.5 kW.
It has 3 charging points.

CS info: ID: 3, initialized. Charge rate: 7.5 kW.
It has 3 charging points.

CS info: ID: 4, initialized. Charge rate: 7.5 kW.
It has 3 charging points.

EV info: ID: 5, destination name: autoshop, journey type: Urban, max_battery: 68, energy consumption rate: 0.22017136695233408, speed: 10, State: Idle.
EV info (Cont'd): Start time: 11, distance goal: 60, soc usage threshold: 34.0, range anxiety 0.5, location: City_D.

EV info: ID: 6, destination name: City C, journey type: InterUrban, max_battery: 49, energy consumption rate: 0.611808866286573, speed: 20, State: Idle.
EV info (Cont'd): Start time: 9, distance goal: 140, soc usage threshold: 24.5, range anxiety 0.5, location: City_D.

EV info: ID: 7, destination name: City C, journey type

In [6]:
# import numpy as np

# def matrix_to_list(matrix):
#     # Convert the matrix to a nested list
#     nested_list = matrix.tolist()

#     # Return the nested list
#     return nested_list
# matrix = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
# nested_list = matrix_to_list(matrix)
# print(nested_list)


Run Model

In [7]:
# model_run.schedule.agents
run_stats = model_run.datacollector.get_model_vars_dataframe()
print(run_stats)

    EVs Charging  EVs Activated  EVs Travelling  EVs Queued  EVs Dead  \
0              0             20               0           0         0   
1              0             20               0           0         0   
2              0             20               0           0         0   
3              0             20               0           0         0   
4              0             20               0           0         0   
5              0             20               0           0         0   
6              0             20               1           0         0   
7              0             20               7           0         0   
8              0             20              10           0         0   
9              0             20              13           0         0   
10             0             20              10           3         0   
11             1             20              13           2         0   
12             2             20               8    

Export results to CSV file

In [ ]:
# today's date as string
date_str = str(datetime.today())
# export csv
model_run.datacollector.get_model_vars_dataframe().to_csv(cfg.DATA_PATH + 'data_' + date_str[0:10] + '_' + str(no_evs) + '_EV_agent_model_output.csv')

### Data import and preprocessing

Data import and Helper functions

In [ ]:
import analysis.helper as helper
import plotly.express as px
import plotly.graph_objs as go
import pandas as pd

In [ ]:
data = pd.read_csv(cfg.DATA_PATH +'modeldatadata_2023-03-19_500_EV_agent_model_output.csv')

data = data.drop(columns=['Unnamed: 0'])

In [ ]:
# data.info()
data.head()

In [ ]:
a = helper.unpack_and_join(data, 'EVs Charge Level')
print(a)

In [ ]:
# # EV charge level per EV per timestep - 20 EVs unpacked
# newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4', 'EVs Charge Level_unpacked_5', 'EVs Charge Level_unpacked_6', 'EVs Charge Level_unpacked_7', 'EVs Charge Level_unpacked_8', 'EVs Charge Level_unpacked_9', 'EVs Charge Level_unpacked_10', 'EVs Charge Level_unpacked_11', 'EVs Charge Level_unpacked_12', 'EVs Charge Level_unpacked_13', 'EVs Charge Level_unpacked_14', 'EVs Charge Level_unpacked_15', 'EVs Charge Level_unpacked_16', 'EVs Charge Level_unpacked_17', 'EVs Charge Level_unpacked_18', 'EVs Charge Level_unpacked_19']]
# newdf.head()

# EV charge level per EV per timestep - 20 EVs unpacked
newdf = a[['EVs Charge Level', 'EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4']]
newdf.head()

### Batching

In [ ]:
from mesa.batchrunner import BatchRunner

EVcounts = (100,500,1000)
cpcounts = (1,2)
tickcounts = (24,48)
model_reporters={'EVs Charged': get_evs_charged,
                'EVs Activated': get_evs_active,
                'EVs Travelling': get_evs_travel,
                'EVs Charge Level': get_evs_charge_level,
                'EVs Currently charging': get_evs_charging,
                'EVs Not Idle': get_evs_not_idle,
                'EOD Battery SOC': get_eod_evs_socs,
                'EVs Destinations': get_evs_destinations,
                }
# parameters = {"no_evs": range(1000,20000,3000), "no_cps": 1}
parameters = {"no_evs": EVcounts, "no_cps": cpcounts, "ticks": tickcounts}
batch_run = BatchRunner(model.EVModel, parameters, max_steps=24, iterations=1, model_reporters= model_reporters) #iterations=1
batch_run.run_all()

In [ ]:
# batch_df = batch_run.get_model_vars_dataframe()

In [ ]:
# print(batch_df)

### Visualisations

Scatter plot

In [ ]:
# plot_data_lines(newdf, 'Timestep', ['EVs Charge Level_unpacked_0', 'EVs Charge Level_unpacked_1', 'EVs Charge Level_unpacked_2', 'EVs Charge Level_unpacked_3', 'EVs Charge Level_unpacked_4'])

### Scrapbook